# SLSLab - Demo

In [1]:
from slslab import *

limit_ships = [10, 61]
limit_locks = [1, 3]
limit_chambers = [1, 3]
traffics = ["(50,50)", "(60,40)", "(70,30)", "(80,20)"]
categories = ["Sophisticated", "NonSophisticated"]
categories_bool = [True, False]
master = {
    "BENDERS": True, # lbbd model
    "MONOLIT": False # monolith
}

lower_bound_stepsize = 10
eta_lower_bounds = range(10,30+lower_bound_stepsize+1,lower_bound_stepsize)
eta_upper_bound_step = 5


ship_length_ranges=[(3500, 5500),(7000, 11000)]
ship_width_ranges=[(475, 650), (950, 1700)]


input_files = []
for k,v in master.items():
    for l in range(limit_locks[0], limit_locks[1] + 1):
        for c in range(limit_chambers[0], limit_chambers[1] + 1):
            for s in range(limit_ships[0], limit_ships[1] + 1, 5):
                for eta in eta_lower_bounds:
                    for index, category in enumerate(categories):
                        for traffic in traffics:
                            for ship_length_range,ship_width_range in zip(ship_length_ranges,ship_width_ranges):
                                c_path = f"{c}_chambers"
                                ls_path = f"{l}lock_{s}ship"
                                eta_path = f"{eta}_{eta+eta_upper_bound_step}eta"
                                ship_dim = f"sl_{ship_length_range[0]}_{ship_length_range[1]}_sw_{ship_width_range[0]}_{ship_width_range[1]}"
                                category_path = category
                                traffic_path = "traffic" + traffic
                                data_path = GENERATED_DIR / c_path / ls_path / eta_path / ship_dim / category_path / format_str_tuple(traffic) 
                                #for json_file in input_JSON_list:
                                cfg = InputConfig(
                                    is_master=v,
                                    is_extobj=False,
                                    ship_count=s,
                                    n_locks=l,
                                    chambers_per_lock=c,
                                    ship_distribution_range=unpack_str_tuple(traffic),
                                    is_sophisticated=categories_bool[index],
                                    chamber_width_cm_range=(2000,3000),
                                    chamber_length_cm_range=(18000,25000),
                                    ship_length_cm_range=ship_length_range,
                                    ship_width_cm_range=ship_width_range,
                                    eta_range=(eta,eta+eta_upper_bound_step),
                                    seed=123,
                                )
                                path = data_path / f"{k}_{s}ships.json"
                                input_files.append(path)
                                generator = InstanceGenerator(input_config=cfg)
                                instance = generator.return_instance()
                                save_instance_as_json(instance, path)
